# 用于对本月的历史数据按照板块名称进行分组统计，结果保存到result.csv文件

In [4]:
import pandas as pd
from datetime import datetime

In [5]:
def get_data() -> tuple[pd.DataFrame, list]:
    """
    获得股票历史信息，并计算总市值
    """
    # 显示结果
    df = pd.read_csv(
        f"../data/merge_{datetime.now().strftime('%Y-%m-%d')}.csv", parse_dates=['日期'], index_col=0, dtype={"股票代码": object})
    # dates = df.index.unique().sort_values().to_list()
    # print(type(dates[0]))
    # dates = [x.strftime("%Y-%m-%d") for x in dates]
    # 获得当前结果集的日期列表
    # dates_list = [date.strftime('%Y-%m-%d') for date in dates]
    value = pd.read_csv("../data/总股本.csv", index_col=0, dtype={"代码": object})
    value_dict = value['总股本'].to_dict()
    df['总股本'] = df['股票代码'].apply(lambda x: value_dict.get(x))
    df['总市值'] = df['总股本']*df['收盘']

    return df


In [6]:
df = get_data()
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 161314 entries, 2023-03-01 to 2023-03-23
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   开盘      161314 non-null  float64
 1   收盘      161314 non-null  float64
 2   最高      161314 non-null  float64
 3   最低      161314 non-null  float64
 4   成交量     161314 non-null  int64  
 5   成交额     161314 non-null  float64
 6   振幅      161314 non-null  float64
 7   涨跌幅     161314 non-null  float64
 8   涨跌额     161314 non-null  float64
 9   换手率     161314 non-null  float64
 10  股票代码    161314 non-null  object 
 11  股票名称    161314 non-null  object 
 12  板块名称    161314 non-null  object 
 13  总股本     154485 non-null  float64
 14  总市值     154485 non-null  float64
dtypes: float64(11), int64(1), object(3)
memory usage: 19.7+ MB


In [7]:
df.columns

Index(['开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率', '股票代码',
       '股票名称', '板块名称', '总股本', '总市值'],
      dtype='object')

## 按日期、板块名称分组，并统计涨幅大于0和小于0的股票数量

In [8]:
# cur_date="2023-03-01"
# cur_df = df.loc[cur_date]
cur_df = df.copy()
# 按股票名称分组，并统计涨幅大于0和小于0的股票数量
result = cur_df.groupby(['日期', '板块名称'])['涨跌幅'].agg(
    [('涨的数量', lambda x: sum(x > 0)), ('跌的数量', lambda x: sum(x < 0))])
result['涨幅比'] = result['涨的数量']/(result['涨的数量']+result['跌的数量'])*100
result


涨的数量  跌的数量        涨幅比
日期         板块名称                       
2023-03-01 IT服务   120     7  94.488189
           LED     31     7  81.578947
           专业工程    31     3  91.176471
           专业服务    20     6  76.923077
           专业连锁     6     1  85.714286
...               ...   ...        ...
2023-03-23 饮料制造    37     9  80.434783
           饰品       4    11  26.666667
           饲料       5    13  27.777778
           高速公路     5    13  27.777778
           黑色家电     8     2  80.000000

[4896 rows x 3 columns]

## 按日期、板块名称分组，并统计涨幅平均值以及总市值求和

In [9]:
cur_df = df.copy()
value_df = cur_df.groupby(['日期', "板块名称"]).agg(
    {"涨跌幅": "mean", "总市值": "sum"})
value_df


涨跌幅           总市值
日期         板块名称                        
2023-03-01 IT服务  4.119685  1.235165e+12
           LED   1.099000  2.977415e+11
           专业工程  1.343529  3.725464e+11
           专业服务  0.585556  1.848354e+11
           专业连锁  0.792500  6.418736e+10
...                   ...           ...
2023-03-23 饮料制造  0.896809  4.807890e+12
           饰品   -0.560625  1.222653e+11
           饲料   -0.498889  2.795893e+11
           高速公路 -0.145000  2.334328e+11
           黑色家电  0.794545  1.269718e+11

[4896 rows x 2 columns]

## 合并结果

In [10]:
# cur_df = cur_df[(cur_df['总市值'] >= (start_value)*100_000_000)
#                 & (cur_df['总市值'] <= (end_value)*100_000_000)]
# cur_df

final_df = result.join(value_df, on=["日期","板块名称"])
# final_df.dropna(inplace=True, axis=0)
# 将Salary列格式化为亿元
final_df['总市值亿元'] = final_df['总市值'].apply(
    lambda x: '{:.2f}'.format(x/100000000))
final_df


涨的数量  跌的数量        涨幅比       涨跌幅           总市值     总市值亿元
日期         板块名称                                                         
2023-03-01 IT服务   120     7  94.488189  4.119685  1.235165e+12  12351.65
           LED     31     7  81.578947  1.099000  2.977415e+11   2977.42
           专业工程    31     3  91.176471  1.343529  3.725464e+11   3725.46
           专业服务    20     6  76.923077  0.585556  1.848354e+11   1848.35
           专业连锁     6     1  85.714286  0.792500  6.418736e+10    641.87
...               ...   ...        ...       ...           ...       ...
2023-03-23 饮料制造    37     9  80.434783  0.896809  4.807890e+12  48078.90
           饰品       4    11  26.666667 -0.560625  1.222653e+11   1222.65
           饲料       5    13  27.777778 -0.498889  2.795893e+11   2795.89
           高速公路     5    13  27.777778 -0.145000  2.334328e+11   2334.33
           黑色家电     8     2  80.000000  0.794545  1.269718e+11   1269.72

[4896 rows x 6 columns]

## 按照涨跌幅的区间进行统计

In [11]:
# 按涨跌幅统计
x = ["跌停", "跌<-5%",  "-3%>-5%",     "-1-3%",    "平盘",
     "<3%",     "3-5%",   "5%-涨停", "涨停"]

db = df.copy()
# db = df.loc['2023-03-01']
bins = [-20, -10, -5, -3, -0.099, 0.099, 3, 5, 10, 20]
# bins = list(range(-11, 12))
cuts = pd.cut(db['涨跌幅'], bins=bins)

x = ["跌停", "跌<-5%",  "-5<-3%",     "-1<3%",    "平盘",
     "1<3%",     "3-5%",   "5%-涨停", "涨停"]
cuts.categories =x
cuts

日期
2023-03-01      (0.099, 3.0]
2023-03-01      (0.099, 3.0]
2023-03-02      (-5.0, -3.0]
2023-03-02      (-5.0, -3.0]
2023-03-03    (-3.0, -0.099]
                   ...      
2023-03-17    (-3.0, -0.099]
2023-03-20    (-3.0, -0.099]
2023-03-21      (0.099, 3.0]
2023-03-22    (-3.0, -0.099]
2023-03-23    (-3.0, -0.099]
Name: 涨跌幅, Length: 161314, dtype: category
Categories (9, interval[float64, right]): [(-20.0, -10.0] < (-10.0, -5.0] < (-5.0, -3.0] < (-3.0, -0.099] ... (0.099, 3.0] < (3.0, 5.0] < (5.0, 10.0] < (10.0, 20.0]]

In [12]:
pct_chg_list = db.groupby(["日期","板块名称", cuts])['涨跌幅'].count()
pct_chg_list


日期          板块名称  涨跌幅            
2023-03-01  IT服务  (-20.0, -10.0]     0
                  (-10.0, -5.0]      0
                  (-5.0, -3.0]       0
                  (-3.0, -0.099]     6
                  (-0.099, 0.099]    1
                                    ..
2023-03-23  黑色家电  (-0.099, 0.099]    1
                  (0.099, 3.0]       8
                  (3.0, 5.0]         0
                  (5.0, 10.0]        0
                  (10.0, 20.0]       0
Name: 涨跌幅, Length: 44064, dtype: int64

In [13]:
pct_chg_list.index



MultiIndex([('2023-03-01', 'IT服务',  (-20.0, -10.0]),
            ('2023-03-01', 'IT服务',   (-10.0, -5.0]),
            ('2023-03-01', 'IT服务',    (-5.0, -3.0]),
            ('2023-03-01', 'IT服务',  (-3.0, -0.099]),
            ('2023-03-01', 'IT服务', (-0.099, 0.099]),
            ('2023-03-01', 'IT服务',    (0.099, 3.0]),
            ('2023-03-01', 'IT服务',      (3.0, 5.0]),
            ('2023-03-01', 'IT服务',     (5.0, 10.0]),
            ('2023-03-01', 'IT服务',    (10.0, 20.0]),
            ('2023-03-01',  'LED',  (-20.0, -10.0]),
            ...
            ('2023-03-23', '高速公路',    (10.0, 20.0]),
            ('2023-03-23', '黑色家电',  (-20.0, -10.0]),
            ('2023-03-23', '黑色家电',   (-10.0, -5.0]),
            ('2023-03-23', '黑色家电',    (-5.0, -3.0]),
            ('2023-03-23', '黑色家电',  (-3.0, -0.099]),
            ('2023-03-23', '黑色家电', (-0.099, 0.099]),
            ('2023-03-23', '黑色家电',    (0.099, 3.0]),
            ('2023-03-23', '黑色家电',      (3.0, 5.0]),
            ('2023-03-23', '黑色

In [14]:
# db = pd.DataFrame(pct_chg_list)
# db.reset_index(inplace=True,level=[0, 1])
db = pct_chg_list.unstack()
db


涨跌幅              (-20.0, -10.0]  (-10.0, -5.0]  (-5.0, -3.0]  (-3.0, -0.099]  \
日期         板块名称                                                                
2023-03-01 IT服务               0              0             0               6   
           LED                0              0             0               5   
           专业工程               0              0             0               3   
           专业服务               0              0             0               6   
           专业连锁               0              1             0               0   
...                         ...            ...           ...             ...   
2023-03-23 饮料制造               0              0             0               8   
           饰品                 0              0             0              11   
           饲料                 0              0             0              13   
           高速公路               0              0             0              13   
           黑色家电               0              0             0               2   

涨跌幅              (-0.099, 0.099]  (0.099, 3.0]  (3.0, 5.0]  (5.0, 10.0]  \
日期         板块名称                                                           
2023-03-01 IT服务                1            41          45           29   
           LED                 5            28           1            1   
           专业工程                1            28           1            1   
           专业服务                4            14           3            0   
           专业连锁                1             5           0            1   
...                          ...           ...         ...          ...   
2023-03-23 饮料制造                3            33           3            0   
           饰品                  1             4           0            0   
           饲料                  0             5           0            0   
           高速公路                2             4           1            0   
           黑色家电                1             8           0            0   

涨跌幅              (10.0, 20.0]  
日期         板块名称                
2023-03-01 IT服务             4  
           LED              0  
           专业工程             0  
           专业服务             0  
           专业连锁             0  
...                       ...  
2023-03-23 饮料制造             0  
           饰品               0  
           饲料               0  
           高速公路             0  
           黑色家电             0  

[4896 rows x 9 columns]

In [15]:
result = pd.merge(final_df, db, on=["日期",'板块名称'])
result.head(10)


涨的数量  跌的数量        涨幅比       涨跌幅           总市值     总市值亿元  \
日期         板块名称                                                            
2023-03-01 IT服务   120     7  94.488189  4.119685  1.235165e+12  12351.65   
           LED     31     7  81.578947  1.099000  2.977415e+11   2977.42   
           专业工程    31     3  91.176471  1.343529  3.725464e+11   3725.46   
           专业服务    20     6  76.923077  0.585556  1.848354e+11   1848.35   
           专业连锁     6     1  85.714286  0.792500  6.418736e+10    641.87   
           专用设备   135    55  71.052632  0.720102  1.494959e+12  14949.59   
           个护用品     4     8  33.333333 -0.180769  6.866488e+10    686.65   
           中药      38    29  56.716418  0.218933  1.105976e+12  11059.76   
           乘用车      8     1  88.888889  1.413333  9.225317e+11   9225.32   
           乳品       9     9  50.000000  0.038333  3.096878e+11   3096.88   

                 (-20.0, -10.0]  (-10.0, -5.0]  (-5.0, -3.0]  (-3.0, -0.099]  \
日期         板块名称                                                                
2023-03-01 IT服务               0              0             0               6   
           LED                0              0             0               5   
           专业工程               0              0             0               3   
           专业服务               0              0             0               6   
           专业连锁               0              1             0               0   
           专用设备               0              0             2              51   
           个护用品               0              0             0               8   
           中药                 0              0             1              28   
           乘用车                0              0             0               1   
           乳品                 0              0             1               7   

                 (-0.099, 0.099]  (0.099, 3.0]  (3.0, 5.0]  (5.0, 10.0]  \
日期         板块名称                                                           
2023-03-01 IT服务                1            41          45           29   
           LED                 5            28           1            1   
           专业工程                1            28           1            1   
           专业服务                4            14           3            0   
           专业连锁                1             5           0            1   
           专用设备               10           119          11            3   
           个护用品                1             4           0            0   
           中药                  9            35           1            1   
           乘用车                 0             7           1            0   
           乳品                  2             8           0            0   

                 (10.0, 20.0]  
日期         板块名称                
2023-03-01 IT服务             4  
           LED              0  
           专业工程             0  
           专业服务             0  
           专业连锁             0  
           专用设备             1  
           个护用品             0  
           中药               0  
           乘用车              0  
           乳品               0

In [16]:
result.tail()

涨的数量  跌的数量        涨幅比       涨跌幅           总市值     总市值亿元  \
日期         板块名称                                                            
2023-03-23 饮料制造    37     9  80.434783  0.896809  4.807890e+12  48078.90   
           饰品       4    11  26.666667 -0.560625  1.222653e+11   1222.65   
           饲料       5    13  27.777778 -0.498889  2.795893e+11   2795.89   
           高速公路     5    13  27.777778 -0.145000  2.334328e+11   2334.33   
           黑色家电     8     2  80.000000  0.794545  1.269718e+11   1269.72   

                 (-20.0, -10.0]  (-10.0, -5.0]  (-5.0, -3.0]  (-3.0, -0.099]  \
日期         板块名称                                                                
2023-03-23 饮料制造               0              0             0               8   
           饰品                 0              0             0              11   
           饲料                 0              0             0              13   
           高速公路               0              0             0              13   
           黑色家电               0              0             0               2   

                 (-0.099, 0.099]  (0.099, 3.0]  (3.0, 5.0]  (5.0, 10.0]  \
日期         板块名称                                                           
2023-03-23 饮料制造                3            33           3            0   
           饰品                  1             4           0            0   
           饲料                  0             5           0            0   
           高速公路                2             4           1            0   
           黑色家电                1             8           0            0   

                 (10.0, 20.0]  
日期         板块名称                
2023-03-23 饮料制造             0  
           饰品               0  
           饲料               0  
           高速公路             0  
           黑色家电             0

In [17]:
result.describe()

,涨的数量,跌的数量,涨幅比,涨跌幅,总市值,"(-20.0, -10.0]","(-10.0, -5.0]","(-5.0, -3.0]","(-3.0, -0.099]","(-0.099, 0.099]","(0.099, 3.0]","(3.0, 5.0]","(5.0, 10.0]","(10.0, 20.0]"
count,4896.000000,4896.000000,4896.000000,4896.000000,4.896000e+03,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000
mean,14.755310,17.176879,45.972778,-0.075540,5.355241e+11,0.024101,0.354167,1.747958,14.729167,1.672181,12.037377,1.466503,0.750613,0.153799
std,24.010556,26.550851,31.277329,1.392680,8.551270e+11,0.166166,1.217331,4.979200,22.108798,2.709377,18.990585,3.993931,2.421916,0.622229
min,0.000000,0.000000,0.000000,-5.855000,6.339136e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,17.925824,-0.983500,1.154625e+11,0.000000,0.000000,0.000000,3.000000,0.000000,2.000000,0.000000,0.000000,0.000000
50%,7.000000,9.000000,44.186047,-0.069545,2.275138e+11,0.000000,0.000000,0.000000,8.000000,1.000000,6.000000,0.000000,0.000000,0.000000
75%,16.000000,19.000000,73.786408,0.696912,5.564626e+11,0.000000,0.000000,2.000000,17.000000,2.000000,14.000000,1.000000,1.000000,0.000000
max,287.000000,317.000000,100.000000,10.668333,7.055796e+12,2.000000,32.000000,102.000000,240.000000,38.000000,238.000000,90.000000,51.000000,11.000000


## 重命名列

In [18]:
result.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4896 entries, (Timestamp('2023-03-01 00:00:00'), 'IT服务') to (Timestamp('2023-03-23 00:00:00'), '黑色家电')
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   涨的数量             4896 non-null   int64  
 1   跌的数量             4896 non-null   int64  
 2   涨幅比              4896 non-null   float64
 3   涨跌幅              4896 non-null   float64
 4   总市值              4896 non-null   float64
 5   总市值亿元            4896 non-null   object 
 6   (-20.0, -10.0]   4896 non-null   int64  
 7   (-10.0, -5.0]    4896 non-null   int64  
 8   (-5.0, -3.0]     4896 non-null   int64  
 9   (-3.0, -0.099]   4896 non-null   int64  
 10  (-0.099, 0.099]  4896 non-null   int64  
 11  (0.099, 3.0]     4896 non-null   int64  
 12  (3.0, 5.0]       4896 non-null   int64  
 13  (5.0, 10.0]      4896 non-null   int64  
 14  (10.0, 20.0]     4896 non-null   int64  
dtypes: float64(3), int64(11), o

In [19]:
result.reset_index(inplace=True,level=[0,1])
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4896 entries, 0 to 4895
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   日期               4896 non-null   datetime64[ns]
 1   板块名称             4896 non-null   object        
 2   涨的数量             4896 non-null   int64         
 3   跌的数量             4896 non-null   int64         
 4   涨幅比              4896 non-null   float64       
 5   涨跌幅              4896 non-null   float64       
 6   总市值              4896 non-null   float64       
 7   总市值亿元            4896 non-null   object        
 8   (-20.0, -10.0]   4896 non-null   int64         
 9   (-10.0, -5.0]    4896 non-null   int64         
 10  (-5.0, -3.0]     4896 non-null   int64         
 11  (-3.0, -0.099]   4896 non-null   int64         
 12  (-0.099, 0.099]  4896 non-null   int64         
 13  (0.099, 3.0]     4896 non-null   int64         
 14  (3.0, 5.0]       4896 non-null   int64  

In [20]:
x = ["跌停", "跌<-5%",  "-5<-3%",     "-1<3%",    "平盘",
     "1<3%",     "3-5%",   "5%-涨停", "涨停"]
a = result.columns[:8].to_list()
a.extend(x)
# a
result.columns =a
result.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4896 entries, 0 to 4895
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   日期      4896 non-null   datetime64[ns]
 1   板块名称    4896 non-null   object        
 2   涨的数量    4896 non-null   int64         
 3   跌的数量    4896 non-null   int64         
 4   涨幅比     4896 non-null   float64       
 5   涨跌幅     4896 non-null   float64       
 6   总市值     4896 non-null   float64       
 7   总市值亿元   4896 non-null   object        
 8   跌停      4896 non-null   int64         
 9   跌<-5%   4896 non-null   int64         
 10  -5<-3%  4896 non-null   int64         
 11  -1<3%   4896 non-null   int64         
 12  平盘      4896 non-null   int64         
 13  1<3%    4896 non-null   int64         
 14  3-5%    4896 non-null   int64         
 15  5%-涨停   4896 non-null   int64         
 16  涨停      4896 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(11), object(

In [21]:
result.head(10)


,日期,板块名称,涨的数量,跌的数量,涨幅比,涨跌幅,总市值,总市值亿元,跌停,跌<-5%,-5<-3%,-1<3%,平盘,1<3%,3-5%,5%-涨停,涨停
0,2023-03-01,IT服务,120,7,94.488189,4.119685,1.235165e+12,12351.65,0,0,0,6,1,41,45,29,4
1,2023-03-01,LED,31,7,81.578947,1.099000,2.977415e+11,2977.42,0,0,0,5,5,28,1,1,0
2,2023-03-01,专业工程,31,3,91.176471,1.343529,3.725464e+11,3725.46,0,0,0,3,1,28,1,1,0
3,2023-03-01,专业服务,20,6,76.923077,0.585556,1.848354e+11,1848.35,0,0,0,6,4,14,3,0,0
4,2023-03-01,专业连锁,6,1,85.714286,0.792500,6.418736e+10,641.87,0,1,0,0,1,5,0,1,0
5,2023-03-01,专用设备,135,55,71.052632,0.720102,1.494959e+12,14949.59,0,0,2,51,10,119,11,3,1
6,2023-03-01,个护用品,4,8,33.333333,-0.180769,6.866488e+10,686.65,0,0,0,8,1,4,0,0,0
7,2023-03-01,中药,38,29,56.716418,0.218933,1.105976e+12,11059.76,0,0,1,28,9,35,1,1,0
8,2023-03-01,乘用车,8,1,88.888889,1.413333,9.225317e+11,9225.32,0,0,0,1,0,7,1,0,0
9,2023-03-01,乳品,9,9,50.000000,0.038333,3.096878e+11,3096.88,0,0,1,7,2,8,0,0,0


In [22]:
result.to_csv(f"../data/result_{datetime.now().strftime('%Y%m%d')}.csv",index=False)

## 校验一下数据结果

In [23]:
df = pd.read_csv(
    f"../data/result_{datetime.now().strftime('%Y%m%d')}.csv", index_col=0, parse_dates=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4896 entries, 2023-03-01 to 2023-03-23
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   板块名称    4896 non-null   object 
 1   涨的数量    4896 non-null   int64  
 2   跌的数量    4896 non-null   int64  
 3   涨幅比     4896 non-null   float64
 4   涨跌幅     4896 non-null   float64
 5   总市值     4896 non-null   float64
 6   总市值亿元   4896 non-null   float64
 7   跌停      4896 non-null   int64  
 8   跌<-5%   4896 non-null   int64  
 9   -5<-3%  4896 non-null   int64  
 10  -1<3%   4896 non-null   int64  
 11  平盘      4896 non-null   int64  
 12  1<3%    4896 non-null   int64  
 13  3-5%    4896 non-null   int64  
 14  5%-涨停   4896 non-null   int64  
 15  涨停      4896 non-null   int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 650.2+ KB


In [24]:
df.head()

,板块名称,涨的数量,跌的数量,涨幅比,涨跌幅,总市值,总市值亿元,跌停,跌<-5%,-5<-3%,-1<3%,平盘,1<3%,3-5%,5%-涨停,涨停
日期,,,,,,,,,,,,,,,,
2023-03-01,IT服务,120,7,94.488189,4.119685,1.235165e+12,12351.65,0,0,0,6,1,41,45,29,4
2023-03-01,LED,31,7,81.578947,1.099000,2.977415e+11,2977.42,0,0,0,5,5,28,1,1,0
2023-03-01,专业工程,31,3,91.176471,1.343529,3.725464e+11,3725.46,0,0,0,3,1,28,1,1,0
2023-03-01,专业服务,20,6,76.923077,0.585556,1.848354e+11,1848.35,0,0,0,6,4,14,3,0,0
2023-03-01,专业连锁,6,1,85.714286,0.792500,6.418736e+10,641.87,0,1,0,0,1,5,0,1,0


In [25]:
color = ["green", "green", "green", "green",
         "yellow", "red", "red", "red", "red"]


In [26]:
my_df=df.loc["2023-03-01"]
my_df=my_df[my_df['板块名称'] == "IT服务"]
x = ["跌停", "跌<-5%",  "-5<-3%",     "-1<3%",    "平盘",
     "1<3%",     "3-5%",   "5%-涨停", "涨停"]
color = ["green", "green", "green", "green",
         "yellow", "red", "red", "red", "red"]

my=my_df[x].unstack()
my=my.reset_index()
my.columns=['x',"date","y"]
# my
data = pd.DataFrame({"x": my["x"], "y": my["y"], "color": color})
data

,x,y,color
0,跌停,0,green
1,跌<-5%,0,green
2,-5<-3%,0,green
3,-1<3%,6,green
4,平盘,1,yellow
5,1<3%,41,red
6,3-5%,45,red
7,5%-涨停,29,red
8,涨停,4,red


In [27]:
df.groupby(df.index).sum() 

/var/folders/my/x1pz96mj4_l31xr74gb254fh0000gn/T/ipykernel_19649/600881992.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(df.index).sum()


,涨的数量,跌的数量,涨幅比,涨跌幅,总市值,总市值亿元,跌停,跌<-5%,-5<-3%,-1<3%,平盘,1<3%,3-5%,5%-涨停,涨停
日期,,,,,,,,,,,,,,,
2023-03-01,6463,2611,20365.464728,209.220338,1.569646e+14,1569646.50,0,29,125,2325,663,5278,683,307,60
2023-03-02,3247,5853,11391.561706,-65.051826,1.565590e+14,1565590.25,5,128,393,5216,575,2726,249,150,33
2023-03-03,4148,4914,14110.161074,31.274357,1.570793e+14,1570792.58,4,57,251,4438,729,3497,301,175,24
2023-03-06,4330,4799,13273.758459,34.228499,1.569857e+14,1569856.69,4,48,300,4331,580,3417,511,263,27
2023-03-07,841,8545,2952.133741,-524.918945,1.545919e+14,1545918.58,4,274,2167,6077,155,615,101,64,24
2023-03-08,6603,2532,19622.025664,178.772816,1.547700e+14,1547699.71,0,15,124,2301,559,5670,535,223,52
2023-03-09,4382,4730,11879.992111,8.848196,1.546087e+14,1546087.33,0,40,198,4379,596,3653,395,167,49
2023-03-10,1379,7986,3540.142517,-404.376918,1.526168e+14,1526168.37,12,182,796,6918,274,1042,144,76,42
2023-03-13,4332,4830,14507.638933,64.610291,1.539333e+14,1539332.80,2,61,298,4399,502,3437,467,269,57
